<img src=https://brand.uark.edu/_resources/images/UA_Logo_Horizontal.jpg width="400" height="96">

###_Artificial intelligence for image processing and analysis._

# Notebook 2.4 Parallel Processing
---
##### The purpose of this notebook is introduce how to implement comvolution operations on hardware that allows for parallel processing.
##### As previously discussed, the convolution operation is achieved by sliding a kernel across an image. This operation, especially for large images, can take a very long time due to the sliding of the kernel that occurs in _series_ (or one pixel after another).
##### Historically, graphical processing units (GPUs) have been often used for rendering computer graphics at very high rates (60 times a second).  Now GPUs are increasingly utilized for convolution operations, because they allow for many mathermatical operations to be done in _parallel_ (or all at the same time).  
##### In Google Colab, the use of a GPU is free of charge, and only requires a setting to be changed.



### Required packages
---
##### **_Run this code chunk first. If you encounter an error when trying to run code chunks in this notebook, then first try re-running this chunk._**

In [ ]:
# Import all of the necessary packages
import numpy as np
import matplotlib.pyplot as plt
import imageio as io
import torch
import time

# Enabling the use of a GPU
---
##### A GPU is enabled to be used at runtime by selecting
> #### Edit > Notebook Settings
##### on the top of this page.
##### A small window will pop-up, and under
> #### Hardware Acceleration
##### select 
> #### GPU.
##### Then select
> #### Save
##### on the bottom of the pop-up window.
##### The following code chunk can be run to verify that a GPU has been enabled. This code chunk should run almost immediately. If it does not, the you can reset the notebook by selecting 
> #### Runtime > Restart Runtime
##### and refreshing the page.


In [ ]:
try:
  assert torch.tensor(1).cuda() == 1
  print('The GPU has successfully been enabled!')
except:
  print('The GPU has NOT been enabled.')

# Utilizing the GPU
---
##### After enabling the use of a GPU, the time it takes to do a convolution operation can be showcased with the following code chunk.

In [ ]:
# Load red blood cell image from github
url = "https://github.com/aewoessn/outreach-program-2021/blob/main/images/red_blood_cells_small.png?raw=true"
test_image = np.mean(io.imread(url),axis=2) / 255
test_image = torch.tensor(test_image).unsqueeze(0).unsqueeze(1).float()

# Set up arrays to store time values
number_of_filters = 30
filter_size = np.zeros(number_of_filters,dtype='int8')
cpu_time = np.zeros(number_of_filters)
gpu_time = np.zeros(number_of_filters)

# Get cpu timing
number_of_iterations = 1
for j in range(number_of_iterations):
  for i in range(number_of_filters):
    filter_size[i] = i+1
    temp_filter = torch.ones((1,1,filter_size[i],filter_size[i])).float()
    start_time = time.time()
    filtered_image = torch.nn.functional.conv2d(test_image,temp_filter)
    end_time = time.time()
    cpu_time[i] += (end_time - start_time)
cpu_time /= number_of_iterations

# Get gpu timing
test_image = test_image.cuda()
for j in range(number_of_iterations):
  for i in range(number_of_filters):
    temp_filter = torch.ones((1,1,filter_size[i],filter_size[i])).float().cuda()
    start_time = time.time()
    filtered_image = torch.nn.functional.conv2d(test_image,temp_filter)
    end_time = time.time()
    gpu_time[i] += (end_time - start_time)
gpu_time /= number_of_iterations

# Plot findings
fig = plt.figure(figsize=(10,10))
plot1 = plt.plot(filter_size,cpu_time*1000,'r',filter_size,gpu_time*1000,'b')
ax1 = fig.gca()
xlab = ax1.set_xlabel('Size of filter [pixels]').set_fontsize('x-large')
ax1.set_xlim(left=1,right=number_of_filters)
ax1.set_xticks((1,5,10,15,20,25,30))
ylab = ax1.set_ylabel('Time to execute convolution [miliseconds]').set_fontsize('x-large')
legend = ax1.legend(('CPU Time','GPU Time'))

# Finishing up
---
##### **_If you used a GPU and are fininshed with the notebook, please make sure to end your session with Google Colab by selecting:_**
> ### Runtime > Manage sessions
##### **_A window will pop up and you need to locate the current notebook and select:_**
> ### TERMINATE